In [3]:
import pandas as pd

In [6]:
import pandas as pd

# Inicializar DataFrame para los resultados finales
final_results = pd.DataFrame()

chunksize = 10 ** 6  # Ajusta esto según tu memoria disponible
for chunk in pd.read_csv('interactions.csv', chunksize=chunksize):
    # Agrupar y calcular en el chunk
    grouped = chunk.groupby('user_id').agg({
        'book_id': lambda x: list(x),
        'rating': 'mean',
        'is_reviewed': lambda x: sum(x) / len(x)
    }).reset_index()  # Restablecer el índice para mantener user_id como una columna

    # Acumular resultados
    final_results = pd.concat([final_results, grouped])

# Procesar los resultados finales
final_df = final_results.groupby('user_id').agg({
    'book_id': 'first',
    'rating': 'mean',
    'is_reviewed': 'mean'
}).reset_index().rename(columns={
    'book_id': 'read_books', 
    'rating': 'avg_rating', 
    'is_reviewed': 'review_proportion'
})

# Guardar en problem.csv
final_df.to_csv('problem.csv', index=False)

# Processament 

In [1]:
import pandas as pd

In [2]:
problem_df = pd.read_csv('minirawproblems10000.csv')
genres_df = pd.read_csv('genresxbook.csv')
books_df = pd.read_csv('books.csv')

### 1. Eliminar 'review_proportion' de rawproblems.csv

In [3]:
problem_df = problem_df.drop('review_proportion', axis=1)

### 2. Juntar les dues bases de dades sobre llibres : books.csv i genresxbook.csv

In [4]:
books_df_complete = pd.merge(books_df, genres_df, on='book_id', how='left')

### 3. Agafar només els llibres que tinguin noms escrits en lletres italiques

In [5]:
books_df_complete = books_df_complete[~books_df_complete['title'].str.contains('[^\x00-\x7F]+', regex=True)]

In [6]:
#Eliminar la informació addicional sobre el llibre que apareixia també inclós en el títol
books_df_complete['title'] = books_df_complete['title'].str.replace('\(.*?\)', '', regex=True)

### 4. Canviar la variable 'read_books' de la base de dades del problema per tal que només contigui llibres que es trobin dins la base de dades de llibres

In [7]:
valid_book_ids = set(books_df_complete['book_id'])

problem_df['read_books'] = problem_df['read_books'].apply(lambda books: [int(book) for book in eval(books) if int(book) in valid_book_ids])

### 5. Filtrar per tenir només els usuaris que han llegit +10 llibres

In [8]:
problem_df = problem_df[problem_df['read_books'].apply(lambda books: len(books) >= 10)]

### 6. Afegir la variable 'counts' que diu quants llibres ha llegit cada usuari
Ara la base de dades de problema només té 'user_id','read_books' i 'avg_rating'

In [9]:
problem_df['counts'] = problem_df['read_books'].apply(lambda books: len(books))

### 7. Afegir la mitjana de pàgines que l'usuari acostuma a llegir

In [10]:
def avg_pages(book_ids, books_df):
    pages = books_df[books_df['book_id'].isin(book_ids)]['num_pages']
    return round(pages.mean())

problem_df['avg_pages'] = problem_df['read_books'].apply(lambda x: avg_pages(x, books_df))

### 8. Afegir la mitjana dels generes que ha llegit


In [12]:

noms_generes = ['Fantasy', 'Fiction', 'Mystery', 'Poetry', 'History', 'Romance', 'Non-fiction', 'Children', 'Young-adult', 'Comics']

def avg_genre(book_ids, generesxbooks, g):
    genre = generesxbooks[generesxbooks['book_id'].isin(book_ids)][g]
    return genre.mean().round(3)

for g in noms_generes:
    problem_df[g] = problem_df['read_books'].apply(lambda x: avg_genre(x, books_df_complete, g))


### 9. Convertir la mitjana dels géneres en un top 10

In [13]:
generes_list = []
for genere in noms_generes:
    generes_list.append(list(problem_df[genere]))

generes_list = list(zip(*generes_list))

In [14]:
top10 = []
for usuari in generes_list:
    percentatges = list(usuari)
    parells = [(valor, idx) for idx, valor in enumerate(percentatges)]
    parells.sort(key=lambda x: x[0], reverse=True)

    for idx, (perc, pos) in zip(range(10, 0, -1),(parells)):
        if perc != 0: percentatges[pos] = idx
        else: percentatges[pos] = 0
    top10.append(percentatges)

top10 = list(zip(*top10))

In [15]:
for nom, genere in zip(noms_generes,top10):
    problem_df[nom] = genere

### 10. Agafar els tres últims llibres que ha llegit i tractar-los com a la recomanació. També agafar la puntuació d'aquests tres llibres

In [16]:
interactions = pd.read_csv('filtered_interactions.csv')
#al fer canvis en la base de dades l'índex ha canviat
problem_df = problem_df.reset_index()
problem_df = problem_df.drop('index', axis=1)

In [17]:
problem_df['recomendations'] = problem_df['read_books'].apply(lambda x: x[-3:])

In [18]:
temp_df = pd.merge(problem_df.explode('recomendations'), interactions, left_on=['user_id', 'recomendations'], right_on=['user_id', 'book_id'])
grouped = temp_df.groupby('user_id').agg(recom_ratings=('rating', list)).reset_index()
problem_df = pd.merge(problem_df, grouped, on='user_id', how='left')

### 11. Canviar l'ordre de les variables per a les classes de python

In [28]:
problem_df = problem_df[['user_id', 'read_books', 'Fantasy', 'Fiction', 'Mystery',
       'Poetry', 'History', 'Romance', 'Non-fiction', 'Children',
       'Young-adult', 'Comics', 'avg_pages', 'counts', 'avg_rating', 'recomendations','recom_ratings']]


In [32]:
problem_df.to_csv('problema.csv')

### 12. Crear un diccionari de book_id : titol

In [198]:
import json
diccionari = {}
for book, title in zip(books_df_complete['book_id'], books_df_complete['title']):
    diccionari[book] = title

with open('books_dict.txt', 'w', encoding='utf-8') as file:
    file.write(str(diccionari))

In [5]:
problem_df = pd.read_csv('problems40000.csv')

problem_30000 = problem_df.sample(30000, random_state = 444)
problem_df.to_csv('problems30000.csv')
problem_20000 = problem_df.sample(20000, random_state = 444)
problem_df.to_csv('problems20000.csv')
problem_10000 = problem_df.sample(10000, random_state = 444)
problem_df.to_csv('problems10000.csv')
problem_5000 = problem_df.sample(5000, random_state = 444)
problem_df.to_csv('problems5000.csv')
problem_1000 = problem_df.sample(1000, random_state = 444)
problem_df.to_csv('problems1000.csv')
problem_5000 = problem_df.sample(500, random_state = 444)
problem_df.to_csv('problems500.csv')
problem_100 = problem_df.sample(100, random_state = 444)
problem_df.to_csv('problems100.csv')
problem_50 = problem_df.sample(50, random_state = 444)
problem_df.to_csv('problems50.csv')

In [1]:
#per quan es vulgui obrir el .txt
with open('books_dict.txt', 'r', encoding='utf-8') as file:
    d = file.read()

eval(d)

{287140: 'Good Harbor',
 89377: "The Devil's Notebook",
 89378: 'Crowner Royal ',
 287142: 'Spirit Lake - Die Legende des Wendigo',
 89379: 'The Passion ',
 828189: 'Investor Behavior: The Psychology of Financial Planning and Investing',
 388676: "Botham's Book of the Ashes: A Lifetime Love Affair with Cricket's Greatest Rivalry",
 257654: 'What Do You Do?',
 257651: "It's Funny Where Ben's Train Takes Him",
 257653: 'Cruelle',
 581162: 'The tale of Jemima puddle-duck',
 54276: 'Bolton Past',
 54270: 'The Serpent and the Rose ',
 268464: 'Hello from Heaven: A New Field of Research-After-Death Communication Confirms That Life and Love Are Eternal',
 598196: 'The Belonging Series ',
 218093: 'Czaropis Tom II ',
 218098: 'The Invaders',
 38569: 'Beautee and the Beast',
 38568: 'The Fifth Columnist ',
 38561: 'Piercing the Reich',
 38563: "CARRIE'S MAIL ORDER HUSBAND",
 38562: "What's for Dinner?",
 38565: 'Family Life',
 38564: 'Combat and Other Shenanigans',
 246205: 'Robots e imperio',


Aquí hem d'extreure un miniproblem.csv de fer una selecció aleatòria de X usuaris de problem_updated.csv.

Des de Linux cal fer el següent:
```bash
head -n 1 problem_updated.csv > miniproblem.csv
tail -n +2 problem_updated.csv | shuf -n X >> miniproblem.csv
```